<img src="images/bulldozer_logo.png" alt="Bulldozer logo" width="200" align="right"/>

# Bulldozer Tutorial (FOSS4G)

Dimitri Lallement (dimitri.lallement@cnes.fr)


# Outline

* Tutorial preparation
* Bulldozer context
* How Bulldozer works?
* Run Bulldozer
* Upcoming works

<img src="images/bulldozer_logo.png" alt="Bulldozer logo" width="200" align="right"/>

# Tutorial preparation

## Preparation

### Requirements

- python>=3.8
- python-venv or virtualenv
- web access

### Environment setup

It is recommended to install **Bulldozer** into a virtual environment, like [conda](https://docs.conda.io/en/latest/) or [virtualenv](https://virtualenv.pypa.io/en/latest/).  
Environment setup with `virtualenv`:
```sh
python -m venv bulldozer_venv
source bulldozer_venv/bin/activate
```

## Installation

### Using Github

```sh
# Clone the project
git clone https://github.com/CNES/bulldozer.git
cd bulldozer/

# Install the library
pip install .
```

*Install check: When you run `bulldozer --version` you should get `v1.0.0`*

## Jupyter notebook preparation

Install Jupyter packages in previous virtualenv: 

```
pip install notebook rise bokeh
```

Build Jupyter kernel:
```
python -m ipykernel install --sys-prefix --name=bulldozer-kernel --display-name=bulldozer-kernel
```

Jupyter environnement: 
``` 
jupyter notebook
```

<img src="images/bulldozer_logo.png" alt="Bulldozer logo" width="200" align="right"/>

# Bulldozer context

## Bulldozer, a DTM extraction tool

<img src="images/result_overview.gif" alt="Bulldozer overview" width="300" align="right"/>

**Bulldozer** is designed as a pipeline of standalone functions that aims to extract a *Digital Terrain Model* (DTM) from a *Digital Surface Model* (DSM) without any exogenous data.  

Main goals: 

- offer an DTM extraction tool that doesn't require any other data (land cover map, water mask, etc.)
- process large scale DSM (scalability and stability)
- set of standalone functions: you can use the functions without running the full pipeline
- compatible with any raster format DSM
- easy to install, easy to run

*Be aware that Bulldozer is new and still in devlopment. Feel free to communicate your needs to the team :)*

License:  Apache-2.0

Links:

- Repository: [github.com/cnes/bulldozer/](https://github.com/cnes/bulldozer/)
- ISPRS Publication: [D. Lallement, P. Lassalle, Y. Ott, R. Demortier, and J. Delvit, 2022. BULLDOZER: AN AUTOMATIC SELF-DRIVEN LARGE SCALE DTM EXTRACTION METHOD FROM DIGITAL SURFACE MODEL. ISPRS - International Archives of the Photogrammetry, Remote Sensing and Spatial Information Sciences.](https://www.int-arch-photogramm-remote-sens-spatial-inf-sci.net/XLIII-B2-2022/409/2022/)

## Projects context

Bulldozer was initially designed by the CNES in the AI4GEO project to handle large scale CARS DSM.  
But now we aim to address more use cases like the DSM generated in the IGN Lidar HD project for example.

<img src="images/logo_cars.png" alt="CARS logo" width="200" align="left"/>
<img src="images/AI4GEO_logo.png" alt="AI4GEO logo" width="200" align="left"/>
<img src="images/logo_lidarHD.png" alt="LidarHD logo" width="200" align="left"/>

### Authors 

* Dimitri Lallement <dimitri.lallement@cnes.fr>
* Pierre Lassalle <pierre.lassalle@cnes.fr>
* Yanncik Ott <yannick.ott@cnes.fr>

See [Authors.md](https://raw.githubusercontent.com/CNES/bulldozer/master/AUTHORS.md) for full contributions in Github repository.

### Contributions

To do a bug report or a contribution, see the [**Contribution Guide**](https://raw.githubusercontent.com/CNES/bulldozer/master/CONTRIBUTING.md).  

For any help or suggestion, feel free to contact the authors.

### Glossary

<img src="images/dsm_dtm_dhm_illustration.png" alt="DEMs illustration" width="300" align="right"/>

**DEM:** Digital Elevation Model. Usually means all elevation models in raster: DSM, DTM,…

**DSM:** Digital Surface Model. Represents the earth’s surface and includes all objects on it. 

**DTM:** Digital Terrain Model. Represents bare ground surface without any objects like trees or buildings.

**DHM:** Digital Height Model. Represents the elevation of the foreground.

<img src="images/bulldozer_logo.png" alt="Bulldozer logo" width="200" align="right"/>

# How Bulldozer works ?

## Main concept

**Bulldozer** is based on a modified version of the multi-scale Drap Cloth principle: 
<img src="images/Bulldozer_concept.png" alt="Bulldozer concept" width="600" align="center"/>

## Drap Cloth method

<img src="images/drap_cloth_simulation.png" alt="DrapCloth method" width="600" align="center"/>

## Tension process
<img src="images/tension.png" alt="Tension process" width="600" align="center"/>

## Drap Cloth limits

<img src="images/noisy_dsm.png" alt="Noisy DSM" width="450" align="right"/>

- Nodata handling
- Noisy input DSM
- No landscape adaptation
- No consideration of the input DSM (resolution, size, etc.)
- Scalability and stability
- Computational performances



## Bulldozer target

- Large scale DTM
- Without exogenous data
- Handles both photogrammetric and Lidar DSM
- Perfoms on noisy DSM
- Low complexity O(n) and scalable
- Outperforms existing methods
- Adapts to hardware ressources
- Accessible through a Python API
- Adaptative output resolution

## Bulldozer compared to DrapCloth

<img src="images/result.png" alt="Tension process" width="600" align="center"/>

## Bulldozer pipeline

**Bulldozer** is designed as a pipeline of standalone functions that aims to extract a *Digital Terrain Model* (DTM) from a *Digital Surface Model* (DSM).  
But you can also use one of the following function without running the full pipeline:
* **DSM preprocessing**
    * **Nodata extraction:** a group of methods to differentiate and extract nodata related to failed correlations during the DSM computation and those of the image border
    * **Disturbed areas detection:** a method to locate disturbed areas. These noisy areas are mainly related to areas in which the correlator has incorrectly estimated the elevation (water or shadow areas).
* **DTM extraction**
    * **DTM computation:** the main method that extracts the DTM from the preprocessed DSM.
* **DTM postprocessing**
    * **Pits detection:** a method to detect pits in the provided raster and return the corresponding mask.
    * **Pits filling:** a method to fill pits in the generated DTM (or input raster).
    * **DHM computation:** a method to extract the *Digital Height Model* (DHM).


## Interfaces

### Inputs

**Bulldozer** requires a raster format DSM file and an output directory (if the providing path doesn't exist, **Bulldozer** will generates it).  
You can provides those data in the config file or by passing it in the Python API.

### Outputs

**Bulldozer** will generate the `DTM.tif` file, the `DHM.tif` if you have activate the option, a `quality_mask.tif` and a log file.


<img src="images/bulldozer_logo.png" alt="Bulldozer logo" width="200" align="right"/>

## Run Bulldozer

## Quickstart

In order to run **Bulldozer** through the Command Line Interface (CLI), you have to create a configuration file.

You can download the configuration file from the tutorial repository:
```
mkdir /tmp/bulldozer_tutorial/
cd /tmp/bulldozer_tutorial/
wget https://raw.githubusercontent.com/cars-cnes/discover-cnes-3d-tools/gh-pages/data_turkey/config_bulldozer_pre_event.yaml
```

Run **Bulldozer**: 
```
bulldozer --conf config_bulldozer_pre_event.yaml
```
Done!


*Warning: Internet needed to download demo data.*

## Configuration file

Let's take a look at the configuration file structure.  
The file is composed of four parts.  
The first one is the main part (the only required elements in the config file):
```yaml
#-------------------------#
#       Parameters        #
#-------------------------#
# Input DSM path (expected format: "<folder_1>/<folder_2>/<file>.<[tif/tiff]>")
dsmPath : "outputs_cars_pre_event/cars_dsm_pre_event.tif"
# Output directory path (if the directory doesn't exist, create it)
outputDir : "outputs_bulldozer_pre_event"
```

## Configuration file

The second part is the options:
```yaml
#-------------------------#
#         Options         #
#-------------------------#
# If True, generates the DHM (DSM - DTM) in the output directory 
generateDhm : True
# If null, the output DTM resolution will be the same as the input DSM (float value in meter)
outputResolution : null
# If null, bulldozer will use the maximum number of available CPU on your system
nbMaxWorkers : 16
```

## Configuration file

The third part is for the experimented users. Let see that:
```yaml
#-------------------------#
#    Advanced settings    #
#-------------------------#

# Foreground max object width (in meter)
maxObjectWidth : 16
# If True, this flag allows snapping DTM values above the DSM to the DSM values in order to satisfy the property that DTM is always below the DSM.
checkIntersection: False
# Minimum valid height value, None or a float value. If null this parameter is ignored
minValidHeight: 0.0
# If null, the nodata value will be extracted from the input DSM (float value or nan)
noData : null
# If True, keep the intermediate results
developperMode : False
```

## Configuration file

The last part is the core settings. We suggest to not modify those values:
```yaml
#-------------------------#
# Bulldozer core settings #
#-------------------------#
# /!\ Modify those data at your own risk (it is suggested to keep the default values) /!\

# DetectDisturbedAreasAndFill part
slopeThreshold : 2.0
# numnumber of evaluated axis. Vertical and horizontal if true else vertical, horizontal and diagonals.
fourConnexity : True

# DtmExtraction part
uniformFilterSize : 3
preventUnhookIter : 10
numOuterIter : 50
numInnerIter : 10
mpTileSize : 1500
```

## Bulldozer Python API


You can run **Bulldozer** with Python API using the config file:
```python
from bulldozer.pipeline.bulldozer_pipeline import dsm_to_dtm

dsm_to_dtm(config_path="conf/configuration_template.yaml")
 ```
Or by providing directly the input parameters (missing parameters will be replaced by default values):
 ```python
 from bulldozer.pipeline.bulldozer_pipeline import dsm_to_dtm
 # Example with a specific number of workers
 dsm_to_dtm(dsm_path="<input_dsm.tif>", output_dir="<output_dir>", nb_max_workers=16)
```

## Advanced usage

<img src="images/noise_detection.png" alt="disturbed areas mask" width="300" align="right"/>

As we mentioned before, **Bulldozer** can also be used to launch in standalone the functions that compose the pipeline.  
It can be helpfull if you want to detect disturbed areas (shadow, water, etc.) in a DSM, fill pits, etc. 

For each step of the pipeline you can follow one of the tutorial notebook:
* [Running Bulldozer (full pipeline)](https://github.com/CNES/bulldozer/tree/master/docs/notebooks/0_bulldozer_pipeline.ipynb)
* [Preprocessing standalone functions](https://github.com/CNES/bulldozer/tree/master/docs/notebooks/1_preprocess.ipynb)
* [Extraction step](https://github.com/CNES/bulldozer/tree/master/docs/notebooks/2_DTM_extraction.ipynb)
* [Postprocessing standalone functions](https://github.com/CNES/bulldozer/tree/master/docs/notebooks/3_postprocess.ipynb)

<img src="images/bulldozer_logo.png" alt="Bulldozer logo" width="200" align="right"/>

## What's next?

## Upcoming works

- Interface harmonisation (very soon!)
- External stability guarantee
- Full mkdocs documentation
- Publish unit tests
- Try to share the QGIS module through the marketplace
- Upgrade pits and disturbed areas detection